In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


documents = SimpleDirectoryReader("data/Books").load_data(show_progress=True)

# Defina um caminho de cache personalizado
cache_folder = "C:/Users/claud/Desktop/embedding_model"


# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5", cache_folder=cache_folder)




index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True
)




c:\Users\claud\Desktop\IC_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]c:\Users\claud\Desktop\IC_RAG\.venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Generating embeddings: 100%|██████████| 496/496 [00:09<00:00, 53.76it/s]


In [2]:
from llama_index.llms.huggingface import HuggingFaceLLM


def messages_to_prompt(messages):
    prompt = ""
    system_found = False
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}<|end|>\n"
            system_found = True
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}<|end|>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}<|end|>\n"
        else:
            prompt += f"<|user|>\n{message.content}<|end|>\n"

    # trailing prompt
    prompt += "<|assistant|>\n"

    if not system_found:
        prompt = (
            "<|system|>\nYou are a helpful AI assistant.<|end|>\n" + prompt
        )

    return prompt


llm = HuggingFaceLLM(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    model_kwargs={
        "trust_remote_code": False,
    },
    generate_kwargs={"do_sample": True, "temperature": 0.1},
    tokenizer_name="microsoft/Phi-3-mini-4k-instruct",
    query_wrapper_prompt=(
        "<|system|>\n"
        "You are a helpful AI assistant.<|end|>\n"
        "<|user|>\n"
        "{query_str}<|end|>\n"
        "<|assistant|>\n"
    ),
    messages_to_prompt=messages_to_prompt,
    is_chat_model=True,
)



c:\Users\claud\Desktop\IC_RAG\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\claud\Desktop\IC_RAG\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\claud\Desktop\IC_RAG\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\claud\Desktop\IC_RAG\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarni

In [3]:
Settings.llm = llm
query_engine = index.as_query_engine(show_progress=True, streaming=True)

In [4]:

streaming_response = query_engine.query("What is the best activation function for neural networks?")
streaming_response.print_response_stream()

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


The best activation function for neural networks depends on the specific problem and the characteristics of the data. However, ReLU (Rectified Linear Unit) activation function is commonly used due to its simplicity and efficiency. It helps avoid the vanishing gradient problem and allows for faster convergence during training. Other activation functions like ELU (Exponential Linear Unit) and SELU (Scaled Exponential Linear Unit) also have their advantages, such as self-normalizing properties and faster convergence rates. Ultimately, the choice of activation function should be based on the specific requirements of the neural network and the problem being solved.